# CIFAR-10 Classification with CNN

In [1]:
import torch
import torchvision
from torchvision import datasets

from torch import nn
from torch import optim

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

import math
import numpy as np
import matplotlib.pyplot as plt

import neural_nets
import cnn_utils as util

In [2]:
torch.manual_seed(42)

## Import Data

### Download Dataset

In [3]:
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Files already downloaded and verified
Files already downloaded and verified


### Create Loaders

This step is needed in order to automate the loading of images with set batch size.

In [4]:
batch_size = 256

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=batch_size,
                                          shuffle=True)

### Set general parameters

In [6]:
input_dimensions = (32,32)
target_classes = 10

# Model Optimization

### Training hyperparameters

In [7]:
num_epochs = 25
learning_rate = 0.001

## 4: Fully Connected Layers

Having tested multiple architectures for the convolutional layers, the effect of multiple fully connected layers will be tested.

In [8]:
fulcon_params = [[32, 64], [64, 64], [64, 128], [128, 128], [32, 64, 128], [64, 128, 256], [64, 64, 128]]

#### Training

In [9]:
for fci in range(len(fulcon_params)):
    fc = fulcon_params[fci].copy()
    cnn = neural_nets.CNN(n_conv_layers=6,
                          filters=[64, 64, 128, 128, 256, 256],
                          kernel=[3, 3, 3, 3, 3, 3],
                          activation=['relu', 'relu', 'relu', 'relu', 'relu', 'relu'],
                          norm=[False, False, False, False, False, False],
                          pool=[0, 2, 0, 2, 0, 2],
                          input_channels=3,
                          fully_connected=fc,
                          input_dims=input_dimensions,
                          classes=target_classes)
    cross_entropy = nn.CrossEntropyLoss()
    adam = optim.Adam(cnn.parameters(), lr=learning_rate)
    %time losses, accuracies = util.train_loop(cnn, cross_entropy, adam, train_loader, test_loader, num_epochs, 0)

    print("Max accuracy:", max(accuracies), "Epoch:", np.argmax(accuracies)+1, "FC Params:", fc)
    print(76*"-")

CPU times: user 10h 34min 8s, sys: 1h 11min 19s, total: 11h 45min 27s
Wall time: 1h 30min 56s
Max accuracy: 0.7939000129699707 Epoch: 22 FC Params: [4096, 32, 64, 10]
----------------------------------------------------------------------------
CPU times: user 9h 21min 4s, sys: 1h 13min 36s, total: 10h 34min 40s
Wall time: 1h 25min 35s
Max accuracy: 0.7878000140190125 Epoch: 11 FC Params: [4096, 64, 64, 10]
----------------------------------------------------------------------------
CPU times: user 8h 50min 19s, sys: 1h 7min 26s, total: 9h 57min 45s
Wall time: 1h 22min 25s
Max accuracy: 0.7843999862670898 Epoch: 19 FC Params: [4096, 64, 128, 10]
----------------------------------------------------------------------------
CPU times: user 8h 58min 17s, sys: 1h 8min 51s, total: 10h 7min 8s
Wall time: 1h 22min 49s
Max accuracy: 0.7524999976158142 Epoch: 14 FC Params: [4096, 128, 128, 10]
----------------------------------------------------------------------------
CPU times: user 10h 7min 17